In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, roc_auc_score ,roc_curve,auc
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.linear_model import LogisticRegression

In [ ]:
# Import dataset
X_train = pd.read_excel('data/X_train.csv')
X_test = pd.read_excel('data/X_test.csv')
y_train = pd.read_excel('data/y_train.csv')
y_test = pd.read_excel('data/y_test.csv')

In [ ]:
# Check class distribution
y_train['repurchase_again'].value_counts()

# Logistic Regression

In [ ]:
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

In [ ]:
kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=seed)
pred_test_full =0
cv_score =[]
i=1
for train_index,test_index in kf.split(X_train,y_train):
    print('{} of KFold {}'.format(i,kf.n_splits))
    xtr,xvl = X.loc[train_index],X.loc[test_index]
    ytr,yvl = y.loc[train_index],y.loc[test_index]
    
    #model
    lr = LogisticRegression()
    lr.fit(xtr,ytr)
    score = roc_auc_score(yvl,lr.predict(xvl))
    print('ROC AUC score:',score)
    cv_score.append(score)    
    pred_test = lr.predict_proba(x_test)[:,1]
    pred_test_full +=pred_test
    i+=1

# SVM

# Naive Bayes

In [ ]:
nb = MultinomialNB()
nb.fit(X_train_dtm_c, y_train)
y_pred_class_train_c = nb.predict(X_train_dtm_c)
y_pred_class_test_c = nb.predict(X_test_dtm_c)

In [ ]:
print('--- Train ---')
print('Accuracy:', metrics.accuracy_score(y_train, y_pred_class_train_c))
print('Precision:', metrics.precision_score(y_train, y_pred_class_train_c))
print('Recall:', metrics.recall_score(y_train, y_pred_class_train_c))
print('ROC_AUC:', metrics.roc_auc_score(y_train, y_pred_class_train_c))

print('\nConfusion Matrix:')
print(metrics.confusion_matrix(y_train, y_pred_class_train_c))


print('\n--- Test ---')
print('Accuracy:', metrics.accuracy_score(y_test, y_pred_class_test_c))
print('Precision:', metrics.precision_score(y_test, y_pred_class_test_c))
print('Recall:', metrics.recall_score(y_test, y_pred_class_test_c))
print('ROC_AUC:', metrics.roc_auc_score(y_test, y_pred_class_test_c))

print('\nConfusion Matrix:')
print(metrics.confusion_matrix(y_test, y_pred_class_test_c))